In [3]:
import numpy as np
import os

import dataserver as ds
from  model import Model



In [1]:
%load_ext autoreload
%autoreload 2


In [ ]:
SEQ_LEN      = 64
N_FEATURES   = 8
Z_DIM        = 32

dataroot  = "data/session_2"
modelroot = "models/session_2r"
suits     = [1]
sources   = "load"

dataserver = ds.Dataserver(SEQ_LEN , N_FEATURES, suits, sources, dataroot = dataroot)

model = Model(suits[0], SEQ_LEN , N_FEATURES, Z_DIM, modelroot, None)
model.load(modelroot)


In [ ]:
encodings = []
i = 0
while i < (dataserver.get_size(suits[0]) - SEQ_LEN):
    print(f"Loop {i}")

    sample = dataserver.get_sequence(suits[0], i, SEQ_LEN)
    sample = sample.reshape((SEQ_LEN, 1, N_FEATURES))

    encoding = model.get_latents(sample)
    encoding = encoding.squeeze()

    encodings.append(encoding)

    i += SEQ_LEN

encodings = np.stack(encodings, axis = 0)

In [ ]:
from PIL import Image, ImageColor
from IPython.display import display
from scipy.special import softmax
import math

print(encodings.shape)

print( np.min(encodings), np.max(encodings))


def sigmoid(x):
  return 1 / (1 + np.exp(-x))

image = sigmoid(encodings)
#
std    = np.std(image, axis=0)
mean   = np.mean(image, axis=0)


In [ ]:

# colors = [ 
#         "#47b1b9",
#         "#499dbb",
#         "#4a8abd",
#         "#4c76be",
#         "#4e62c0",
#         "#504fc1",
#         "#6751c3",
#         "#7e52c5"]

colbase = np.array([
    ImageColor.getcolor("#47b1b9", "HSV"),
    ImageColor.getcolor("#499dbb", "HSV"),
    ImageColor.getcolor("#4a8abd", "HSV"),
    ImageColor.getcolor("#4c76be", "HSV"),
    ImageColor.getcolor("#4e62c0", "HSV"),
    ImageColor.getcolor("#504fc1", "HSV"),
    ImageColor.getcolor("#6751c3", "HSV"),
    ImageColor.getcolor("#7e52c5", "HSV")
])


def lerp(t, x, y):
    return x + (y - x ) * t
 


v = image.reshape((image.shape[0], image.shape[1], 1))
s = std.repeat(encodings.shape[0]).reshape((encodings.shape[0], encodings.shape[1], 1) ,  order='F')
h = mean.repeat(encodings.shape[0]).reshape((encodings.shape[0], encodings.shape[1], 1),  order='F')

h = lerp(h, 0.0, 1.0)
s = lerp(s, 0.0, 0.0)
v = lerp(v, 0.0, 1.0)


img = np.concatenate((h,s,v), axis=-1)
img = np.clip((img * 255).astype(np.uint8),0,255)

img = Image.fromarray(img, mode="HSV" ).convert("RGB")
img = img.rotate(90, expand=True)
img = img.resize([1920 * 2, 1080], resample=Image.NEAREST)
img.save("figures/encodings_2.jpg")

display(img)